In [1]:
import numpy as np
import pandas as pd
from sunpy.net import hek

In [13]:
client = hek.HEKClient()
start_time = pd.to_datetime('2014/01/01 00:00')
end_time = pd.to_datetime('2014/02/01 00:00')

       

In [10]:
fl_SSW = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()),
                                  hek.attrs.EventType('FL'))
flare = pd.DataFrame(flare_events)

In [11]:
ef_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('EF'))
ef = pd.DataFrame(ef_events)

In [4]:
ar_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('AR'))
ar = pd.DataFrame(ar_events)

In [5]:
cme_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('CE'))
cme = pd.DataFrame(cme_events)

In [6]:
sg_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('SG'))
sg = pd.DataFrame(sg_events)

In [14]:
fe_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('FE'))
fe = pd.DataFrame(fe_events)

In [5]:
# fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),
#                                                  end_time.isoformat()),
#                                   hek.attrs.EventType('FL'),
#                                   hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')
# fl_SSW= client.query(hek.attrs.Time(start_time.isoformat(),
#                                                  end_time.isoformat()),
#                                   hek.attrs.EventType('FL'),
#                                   hek.attrs.FRM.Name == 'SSW Latest Events')

In [ ]:
def determine_wavelengths(index, index2 = None):
    is_131 = 0
    is_171 = 0
    is_193 = 0
    is_211 = 0
    is_304 = 0
    is_355 = 0
    is_94 = 0
    if index2 != None:
        if fl_detective['obs_channelid'].values[index] or fl_detective['obs_channelid'].values[index2] == 131:
            is_131 = 1
        if fl_detective['obs_channelid'].values[index] or fl_detective['obs_channelid'].values[index2] == 171:
            is_171 = 1
        if fl_detective['obs_channelid'].values[index] or fl_detective['obs_channelid'].values[index2] == 193:
            is_193 = 1
        if fl_detective['obs_channelid'].values[index] or fl_detective['obs_channelid'].values[index2] == 211:
            is_211 = 1
        if fl_detective['obs_channelid'].values[index] or fl_detective['obs_channelid'].values[index2] == 304:
            is_304 = 1
        if fl_detective['obs_channelid'].values[index] or fl_detective['obs_channelid'].values[index2] == 355:
            is_355 = 1
        if fl_detective['obs_channelid'].values[index] or fl_detective['obs_channelid'].values[index2] == 94:
            is_94 = 1
    else:
        if fl_detective['obs_channelid'].values[index] == 131:
            is_131 = 1
        if fl_detective['obs_channelid'].values[index] == 171:
            is_171 = 1
        if fl_detective['obs_channelid'].values[index] == 193:
            is_193 = 1
        if fl_detective['obs_channelid'].values[index] == 211:
            is_211 = 1
        if fl_detective['obs_channelid'].values[index] == 304:
            is_304 = 1
        if fl_detective['obs_channelid'].values[index] == 355:
            is_355 = 1
        if fl_detective['obs_channelid'].values[index] or == 94:
            is_94 = 1
    return is_131, is_171, is_193, is_211, is_304, is_355, is_94


In [ ]:
def fl_duplicates_detective(delta_t_max, distance_max, search_index):
    delete_index = []
    i = 0
    detective_start = fl_detective['event_starttime']
    detective_position = map(lambda x: wkt.loads(x), fl_detective['hpc_coord'])
    associated_fl = []
    is_associated_fl = [0 for n in range(flare_set.shape[0])]
    is_131 = 0
    is_171 = 0
    is_193 = 0
    is_211 = 0
    is_304 = 0
    is_355 = 0
    is_94 = 0
    for elem in detective_start:
        fl_match = []
        is_131, is_171, is_193, is_211, is_304, is_355, is_94 = determine_wavelengths(i)
        for j in range(1, search_index):
            i2 = i+j
            if fl_detective['obs_channelid'].values[i] != fl_detective['obs_channelid'].values[i2]:
                if detective_start[i2]-elem <= delta_t_max:
                    if detective_position[i].distance(detective_position[i2]) <= distance_max:
                        delete_index.append(i2)
                        fl_match.append(fl_detective['SOL_standard'].values[i2])
                        is_131, is_171, is_193, is_211, is_304, is_355, is_94 = determine_wavelengths(i, i2)
                    else: fl_match.append([0])
                else: fl_match.append([0])
            else: fl_match.append([0])
    k = 0
    for elem in associated_fl:
        if elem!=[0]:
            is_associated_flare[k] = 1
        k+=1
    fl_detective.drop(delete_index)         
        
    return fl_detective     

In [5]:
delta_t_max = 10 #[minutes]
distance_max = 5 #[arcsec, HPC]
keywords_SSW = ['SOL_standard', 'event_starttime', 'event_endtime', 'event_peaktime', 'hpc_coord', 
                'hpc_bbox', 'fl_goescls']

def fl_duplicates_SSW(delta_t_max, distance_max):
    delete_index = []
    i = 0
    SSW_start = fl_SSW['event_starttime']
    SSW_position = map(lambda x: wkt.loads(x), fl_SSW['hpc_coord'])
    
    for elem in SSW_start:
        j = 1
        is_duplicate = True
        while is_duplicate == True:
            is_duplicate = False
            if fl_SSW['fl_goescls'].values[i] == fl_SSW['fl_goescls'].values[i+j]:
                if SSW_start[i+j]-elem <= delta_t_max:
                    if SSW_position[i].distance(SSW_position[i+j]) <= distance_max:
                        delete_index.append(i+j)
                        is_duplicate = True
                        j+=1
    fl_SSW.drop(delete_index)         
        
    return fl_SSW     

In [ ]:
def fl_goes_class():

In [ ]:
def create_fl_database(start_time, emd_time, delta_t_max, distance_max):
    fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), 
                                hek.attrs.EventType('FL'), hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')
    fl_SSW= client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), hek.attrs.EventType('FL'), 
                         hek.attrs.FRM.Name == 'SSW Latest Events')
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    fl_detective = fl_duplicates_detective(delta_t_max, distance_max)
    fl_SSW = fl_duplicates_SSW(delta_t_max, distance_max)

    
    

In [3]:
my_list = range(10)
print my_list[(1+2)
             ]

3


In [7]:
#export from HEK SSW, SWPC, & flare detective events separately
#assume dupilcates have already been merged together
def combine_flares_SSW_SWPC():
    for elem in fl_SSW['event_starttime']
    fl_SWPC
    

3807


In [12]:
flare_keywords = list(np.genfromtxt('flare_keywords.csv', delimiter=',', dtype=str))
ef_keywords = list(np.genfromtxt('ef_keywords.csv', delimiter=',', dtype =str))
# print flare_keywords
# print ef_keywords
#to change ordering of columns/delete excess ones: df = df[list_keywords]


                       SOL_standard      event_starttime        event_endtime  \
0    SOL2013-12-31T23:30:45L151C090  2013-12-31T23:30:45  2014-01-01T02:18:45   
1    SOL2013-12-31T23:42:45L233C077  2013-12-31T23:42:45  2014-01-01T01:18:45   
2    SOL2014-01-01T00:54:45L223C079  2014-01-01T00:54:45  2014-01-01T06:42:45   
3    SOL2014-01-01T02:18:45L206C084  2014-01-01T02:18:45  2014-01-01T03:42:45   
4    SOL2014-01-01T02:18:45L223C117  2014-01-01T02:18:45  2014-01-01T03:54:45   
5    SOL2014-01-01T02:30:45L233C103  2014-01-01T02:30:45  2014-01-01T04:54:45   
6    SOL2014-01-01T03:42:45L207C084  2014-01-01T03:42:45  2014-01-01T05:42:45   
7    SOL2014-01-01T07:18:45L232C103  2014-01-01T07:18:45  2014-01-01T09:18:45   
8    SOL2014-01-01T07:18:45L211C104  2014-01-01T07:18:45  2014-01-01T12:30:45   
9    SOL2014-01-01T07:54:45L234C094  2014-01-01T07:54:45  2014-01-01T09:30:45   
10   SOL2014-01-01T07:54:45L215C088  2014-01-01T07:54:45  2014-01-01T11:30:45   
11   SOL2014-01-01T08:06:45L

In [15]:
# flare.to_csv('look_at_flare_data.csv')
# ef.to_csv('look_at_ef_data.csv')
# ar.to_csv('look_at_ar_data.csv')
# cme.to_csv('look_at_cme_data.csv')
# sg.to_csv('look_at_sg_data.csv')
fe.to_csv('look_at_fe_data.csv')

In [ ]:
#final step to export dataframe to csv

# fl.to_csv('flares_database.csv', index = False, columns = flare_keywords)
# ef.to_csv('ef_database.csv', index = False, columns = ef_keywords)
# cme.to_csv('cme_database.csv', index = False, columns = cme_keywords)
# ar.to_csv('ar_database.csv', index = False, columns = ar_keywords)
# sg.to_csv('sg_database.csv', index = False, columns = sg_keywords)
# fe.to_csv('fe_database.csv', index = False, columns = fe_keywords)


SyntaxError: can't assign to operator (<ipython-input-6-7a935e9c5ad0>, line 3)